#  Install package

In [1]:
# # You may need these
# ! pip install openai
# ! pip install langchain
# ! pip install langchain-openai
# ! pip install langchain-core
# ! pip install osmnx
# ! pip install langchain openai --upgrade
# ! pip install itables
# ! pip install leafmap
# ! jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-leaflet
# ! pip install localtileserver 

## May not need these
## ! pip install pyvis
## ! pip install networkx
## ! pip install OSMPythonTools
## ! pip install contextily
## ! pip install matplotlib_scalebar
## ! pip install geojson


# Import package

In [2]:
%load_ext autoreload
%autoreload 2
    
import os
import rasterio
from PIL import Image

import requests
import networkx as nx
import pandas as pd
import geopandas as gpd
# from pyvis.network import Network
from openai import OpenAI
from IPython.display import display, HTML, Code
from IPython.display import clear_output
import matplotlib.pyplot as plt

import base64

import pickle
import sys
import osmnx as ox

import LLM_Find_Constants as constants
import helper

import numpy as np
# from LLM_Find_kernel import Solution

from langchain_openai import ChatOpenAI

# from langchain_core.prompts import ChatPromptTemplate
from time import sleep

OpenAI_key = helper.load_OpenAI_key()

In [3]:
display(HTML("""
<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.1.10/require.min.js"></script>
"""))

sleep(0.1)

from itables import init_notebook_mode
from itables import show
# init_notebook_mode(all_interactive=True)

# Input task and data desciption

## Data source 1: OpenStreetMap

In [4]:
## task_name ='Nigeria_cities'
downloaded_file_name = r'Nigeria_cities.gpkg'
saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
task = rf'''1. Download the city locations of Nigeria; do not download towns.   
2. Save the downloaded data as points, save it at: {saved_fname} 
'''

# task_name ='Nigeria_rivers'
# downloaded_file_name = r'Nigeria_rivers.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the rivers of Nigeria.   
# 2. Save the downloaded data as polylines, save it at: {saved_fname} 
# '''

# task_name ='Nigeria_state_boundary'  # most test failed!!!!! Soloved.
# downloaded_file_name = r'Nigeria_states_boundary.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download all state boundaries of Nigeria.   
# # 2. Save the downloaded data as polygons, save it at: {saved_fname} 
# # '''

# task_name ='World_country_boundary'  # most test failed!!!!! Soloved.
# downloaded_file_name = r'World_country_boundary.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download all country boundaries of the world.   
# # 2. Save the downloaded data as polygons, save it at: {saved_fname} 
# # '''


## task_name ='China_mainland_province_boundary'  # most test failed! solved.
# downloaded_file_name = r'China_mainland_Province_boundary.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download all province boundaries of China mainland.   
# 2. Save the downloaded data as polygons in GeoPackage format at: {saved_fname} 
# '''

## task_name ='OSM_PA_boundary'
# downloaded_file_name = r'PA_boundary.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the administrative boundary of Pennsylvania State, USA.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

## task_name ='OSM_SC_boundary'
# downloaded_file_name = r'SC_boundary.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the administrative boundary of South Carolina State, USA.
# 2. Save the downloaded data in GeoPackage format at: {saved_fname} 
# '''

# task_name ='OSM_PA_hospital'
# downloaded_file_name = r'PA_hospital.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download all hospitals in Pennsylvania, USA.
# 2. Save the downloaded data as points in GeoPackage format at: {saved_fname} 
# '''

## task_name ='SC_hospital'
# downloaded_file_name = r'SC_hospital.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download all hospitals in South Carolina, USA.
# 2. Save the downloaded data as points in GeoPackage at: {saved_fname} 
# '''

## task_name ='OSM_SC_school'
# downloaded_file_name = r'SC_school.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download all schools in South Carolina State, USA.
# 2. Save the downloaded data as points in GeoPackage format at: {saved_fname} 
# '''

# task_name ='OSM_Yulin_River'
# downloaded_file_name = r'Yulin_river.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download all rivers in Yulin, Guangxi, China.
# 2. Save the downloaded data as polylines in GeoPackage format at: {saved_fname} 
# '''

## task_name ='OSM_CA_park'
# downloaded_file_name = r'CA_parks.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download all parks in California, USA, including urban public, recreation, state, and national parks.
# 2. Save the downloaded data as points in GeoPackage format at: {saved_fname} 
# # '''

## task_name ='OSM_USA_university'
# downloaded_file_name = r'USA_universities.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the POIs of all universities, colleges, and other higher education institutions in the USA.
# 2. Save the downloaded data as points in GeoPackage format at: {saved_fname} 
# '''

## task_name ='OSM_State_College_street'
# downloaded_file_name = r'State_College_street.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download all streets of State College, Pennsylvania, USA.
# 2. Save the downloaded data as polylines in GeoPackage format at: {saved_fname} 
# '''

# ## task_name ='OSM_Nigeria_boundary'
# downloaded_file_name = r'Nigeria_boundary.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the administrative boundary of Nigeria.
# 2. Save the downloaded data in GeoPackage format at: {saved_fname} 
# '''

# ## task_name ='OSM_Afghanistan_boundary'
# downloaded_file_name = r'Afghanistan_boundary.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the administrative boundary of Afghanistan.
# 2. Save the downloaded data in GeoPackage format at: {saved_fname} 
# '''


# task_name ='OSM_Nigeria_railway'
# downloaded_file_name = r'Nigeria_railway.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the railway network of Nigeria.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

## task_name ='Wuhan_railway_network'
# downloaded_file_name = r'Wuhan_Railway_network.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the railway network in Wuhan, Hubei, China.
# 2. Save the downloaded data as polylines in GeoPackage format at: {saved_fname} 
# '''
# # Wuhan_railway_network is a difficult case! It succeeded at the beginning, but failed all the time later.
# # The query: area["name"="Wuhan"]["boundary"="administrative"]->.searchArea; is not correct. Need to use "name:en". 
# # Using "Hubei Province" may not return polygons

## task_name ='Qingdao_boundary'
# downloaded_file_name = r'Qingdao_boundary.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the administrative of Qingdao, Shandong, China.
# 2. Save the downloaded data as polygons, save it at: {saved_fname} 
# '''

# task_name ='China_Guangdong_province_boundary'   
# downloaded_file_name = r'China_Guangdong_province_boundary.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the Guangdong province boundaries of China.   
# 2. Save the downloaded data as polygons, save it at: {saved_fname} 
# '''

## task_name ='OSM_coffee_shop_Vietnam'
# downloaded_file_name = r'coffee_shop_Vietnam.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download all the coffee shops in Vietnam.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

## Data source 2:  US Census Bureau administrative boundary

In [5]:
## task_name ='Census_SC_tract'
# downloaded_file_name = r'Census_SC_tract.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download all census tract boundaries in South Carolina, USA.
# 2. Save the downloaded data as polygons in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='Census_SC_blockgroups'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_SC_blockgroups.gpkg'
# if os.path.exists(saved_fname):
#     os.remove(saved_fname)
# task = rf'''1. Download all Census block group boundaries in South Carolina, USA.
# 2. Save the downloaded data as polygons in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='Census_Centre_boundary'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_Centre_boundary.gpkg'
# task = rf'''1. Download the administrative boundary of Centre County of Pennsylvania State, USA from Census Bureau.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

# task_name ='Census_SC_countries_boundary'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_SC_counties_boundary.gpkg'
# task = rf'''1. Download the administrative boundary of all Counties of South Carolina from Census Bureau.
# 2. Save the downloaded data in GeoPackage format, save it at: {saved_fname} 
# '''

# # task_name ='US_Carolinas_tract"
# downloaded_file_name = r'US_Carolinas_tract.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download all the Census tract boundaries of North Carolina and South Carolina in the USA.
# 2. Save the downloaded data in GeoPackage format at: {saved_fname} 
# '''

# # task_name ='US_county_boundary"
# downloaded_file_name = r'US_county_boundary.gpkg'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download all the county boundaries in the USA.
# 2. Save the downloaded data in GeoPackage format at: {saved_fname} 
# '''

## Data source 3:  US Census Bureau demographic variables

In [6]:
# task_name ='Census_SC_counties_population'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_SC_counties_population.csv'
# task = rf'''1. Download latest population for each county in South Carolina.
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

# task_name ='Census_SC_Richland_race_population'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_SC_Richland_race_population.csv'
# task = rf'''1. Download latest population of each race for Richland county in South Carolina, at Census block group level.
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

# task_name ='Census_PA_counties_race_population'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_PA_counties_race_population.csv'
# task = rf'''1. Download latest population by race for all counties in Pennsylvania.
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

# task_name ='Census_US_states_population'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_US_states_population.csv'
# task = rf'''1. Download latest population for all states in USA.   
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

# task_name ='Census_US_states_education_population'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_US_states_education_population.csv'
# task = rf'''1. Download latest population by higher education attainment over 25 for all states in USA, together with the entire population of each state.   
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

# task_name ='Census_US_county_household_income'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_US_county_household_income.csv'
# task = rf'''1. Download the latest median household income data for each county in the USA.   
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

# task_name ='Census_US_county_population_by_race'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Census_US_county_population_by_race.csv'
# task = rf'''1. Download the latest population by race data for each county in the USA.   
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

# # task_name ='States_colledge_popultion'   # difficult to get the correct variable combination. 
# downloaded_file_name = r'States_colledge_popultion.csv'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the population over 25 years old and the population with a college degree or higher at the state level of USA for 2012 and 2022.
# 2. Save the downloaded data in CSV format, save it at: {saved_fname} 
# '''

## task_name ='US_SDOH'   # difficult to get the correct variable combination
# downloaded_file_name = r'US_SDOH.csv'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the social determinators of health, including: 1) population of each race; 2) Median household income; 3) health insurance coverage; 4) Population of speaking only English at home for the population 5 years and over.
# 2. The data should be at the county level in the USA. Year: 2022.
# 3. Save the downloaded data in a CSV file, save it at: {saved_fname} 
# '''

# task_name ='US_County_poverty'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\US_County_poverty.csv'
# task = rf'''1. Download the ratios of income to all poverty level at the county level in the USA. Year: 2022.
# 3. Save the downloaded data in a CSV file, save it at: {saved_fname} 
# '''

## task_name ='Washington_DC_blockgroup_senior_population'
# downloaded_file_name = r'Washington_DC_blockgroup_senior_population.csv'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task_name ='Washington_DC_blockgroup_senior_population'
# task = rf'''1. Download the senior (older than  65) population groups senior  for all Census blockgroups in Washington D.C., USA.   
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

# task_name ='School_enrollment'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\School_enrollment_population.csv'
# task = rf'''1. From Census 2020 data, download the school enrollment by level of school for the population 3 years and over of all Census block groups in San Francisco County, California, USA.   
# 2. Save the downloaded data as CSV files, save it at: {saved_fname} 
# '''

##  Data source 4:  COVID-19 accumulative cases by New York Times

In [7]:
## task_name ='COVID_Richland_SC'
# downloaded_file_name = r'COVID_Richland_SC.csv'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the COVID-19 case data of Richland County in South Carolina, USA. The time is from 2021-01 to 2021-09.   
# 2. Save the downloaded data as a CSV file at: {saved_fname} 
# '''

# task_name ='COVID_PA'
# downloaded_file_name = r'COVID_PA.csv'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the COVID-19 case data of all counties in Pennsylvania, USA. The time is from 2021-10 to 2022-02.   
# 2. Save the downloaded data as a CSV file at: {saved_fname} 
# '''

# task_name ='COVID_PA'
# downloaded_file_name = r'COVID_NJ_NY.csv'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the COVID-19 case data of all counties in Pennsylvania State and New York State, USA. The period is entire 2021.   
# 2. Save the downloaded data as a CSV file at: {saved_fname} 
# '''

## Data source 5:  Weather data

In [8]:
## task_name ='OpenWeather_Columbia'
# downloaded_file_name = r'OpenWeather_Columbia.csv'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the historical weather data of Columbia, South Carolina in May 2024.
# 2. Save the downloaded data in CSV format, save it at: {saved_fname} 
# '''

# # task_name ='OpenWeather_Yulin_Guangxi'
# downloaded_file_name = r'OpenWeather_Yulin_Guangxi.csv'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the historical weather data of Yulin, Guangxi, China, in May 2024.
# 2. Save the downloaded data in CSV format, save it at: {saved_fname} 
# '''

## task_name ='OpenWeather_Cairo'
# downloaded_file_name = r'OpenWeather_Cairo.csv'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the current weather data of Cairo, Egypt.
# 2. Save the downloaded data in CSV format, save it at: {saved_fname} 
# '''

## task_name ='OpenWeather_Kabul'
# downloaded_file_name = r'OpenWeather_Kabul.csv'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the 16-day daily forecast weather data of Kabul, Afghanistan.
# 2. Save the downloaded data in CSV format, save it at: {saved_fname} 
# '''

## Data source 6:  Satellite image (ESRI World Imagery (for export))

In [9]:
# task_name ='FAST_Telescope'
# downloaded_file_name = r'FAST_Telescope_image.tif'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the FAST Telescope (Guizhou, China) satellite image at level 18.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

# # task_name ='Nigeria_image'
# downloaded_file_name = r'Nigeria_image.tif'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the Nigeria satellite image at level 7.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

# # task_name ='Qingdao_image'
# downloaded_file_name = r'Qingdao_image.tif'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the Qingdao, Shandong, China satellite image at level 10.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

# ## task_name ='Crescent_Moon_Spring'
# downloaded_file_name = r'Crescent_Moon_Spring_image.tif'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the Singing-Sand Mountain and Crescent Moon Spring satellite image at level 16.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

## task_name ='Christ the Redeemer'  # this is a point: a difficult case
# downloaded_file_name = r'Christ_the_Redeemer.tif'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the Christ the Redeemer satellite image at level 18.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

# #task_name ='Brasília_image'   # not ready yet
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Brasília_image.tif'
# task = rf'''1. Download the Brasília satellite image at level 6.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

## task_name ='Japan_image'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Japan_image.tif'
# task = rf'''1. Download the Japan satellite image at level 6.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

## task_name ='China'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\China_image.tif'
# task = rf'''1. Download the China satellite image at level 6.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

# task_name ='YellowStone_National_Park'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Yellow_Stone_National_Park_image.tif'
# task = rf'''1. Download the YellowStone National Park satellite image at level 10.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

## task_name ='Hawaii'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Hawaii_image.tif'
# task = rf'''1. Download the Hawaii State satellite image at level 7.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

## task_name ='Honolulu'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Honolulu_image.tif'
# task = rf'''1. Download the Honolulu satellite image at level 12.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

## task_name ='Kennedy_Space_Center_Visitor_Complex'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Kennedy_Space_Center_Visitor_Complex_image.tif'
# task = rf'''1. Download the Kennedy Space Center Visitor Complex satellite image at level 18.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

## task_name ='Hoover_Dam'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Hoover_Dam_image.tif'
# task = rf'''1. Download the Hoover Dam satellite image at level 18.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

## task_name ='Nigeria'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Nigeria_image.tif'
# task = rf'''1. Download the Nigeria satellite image at level 8.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

## task_name ='Afghanistan'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Afghanistan_image.tif'
# task = rf'''1. Download the Afghanistan satellite image at level 8.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

## task_name ='State_college'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\State_college_image.tif'
# task = rf'''1. Download the State College City, PA satellite image at level 12.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

# #task_name = 'Tigard'
# saved_fname = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Tigard_OR_image.tif'
# task = rf'''1. Download the Tigard, OR satellite image at level 14.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

## task_name = "Xiong'an"
# downloaded_file_name = r'Xiong_an.tif'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the Xiong'an New Area, China satellite image at level 12.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

## task_name = "Penn_State_University"
# downloaded_file_name = r'Penn_State_University.tif'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the Pennsylvania State University satellite image at level 15.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

## Data source 7:  OpenTopography

In [10]:
## task_name = "Wuhan_DEM"
# downloaded_file_name = r'Wuhan_DEM.tif'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the 90m resolution DEM of Wuhan, China from SRTMGL3.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

# # task_name = "Chongqing_DEM"
# downloaded_file_name = r'Chongqing_DEM.tif'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the 30m resolution DEM of Chongqing, China from SRTMGL1.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

# # task_name = "Hawaii_DEM"
# downloaded_file_name = r'Hawaii_DEM.tif'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the 90m resolution DEM of Hawaii, USA from SRTMGL3.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''


## task_name = "Lhasa, China_DEM"
# downloaded_file_name = r'Lhasa_DEM.tif'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the 30m resolution DEM of Lhasa, China, from COP30.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''


# # task_name = "Iceland"
# downloaded_file_name = r'Iceland_DEM.tif'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the 30m resolution DEM of Iceland_DEM from EU_DTM.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

## task_name = "Mount Everest"
# downloaded_file_name = r'Everest_DEM.tif'
# saved_fname = os.path.join(os.getcwd(), "Downloaded_Data", downloaded_file_name)
# task = rf'''1. Download the 30m resolution DEM of this region [south:27.82, west:86.73, north:28.17, east:87.13] from AW3D30.
# 2. Save the downloaded data in tiff format, save it at: {saved_fname} 
# '''

# Fetching data

## Create the model

In [11]:
if os.path.exists(saved_fname):
    os.remove(saved_fname)

save_dir = os.path.join(os.getcwd(), "Downloaded_Data")
os.makedirs(save_dir, exist_ok=True)

model_name = r'gpt-4o'
# model_name = r'gpt-4'
# model_name = r'gpt-4-turbo'
# model_name = r'gpt-3.5-turbo'  # gpt-4-turbo

model = ChatOpenAI(api_key=OpenAI_key, model=model_name, temperature=1)

## Select the data source

In [12]:
# %load_ext autoreload
# %autoreload 2
import helper
    
source_select_prompt_str = helper.create_select_prompt(task=task)

print(source_select_prompt_str)

Your role: A professional Python programmer in geographic information science (GIScience). You have worked on GIScience for more than 20 years and know every detail and pitfall when collecting data and coding. You know which websites you can get suitable spatial data and know the methods or tricks to download data, such as OpenStreetMap, Census Bureau, or various APIs. You are also experienced in processing the downloaded data, including saving them in suitable formats, map projections, and creating detailed and useful meta-data.
 
Your mission: select a suitable data source from the given list to download the requested geo-spatial data for this task: 1. Download the city locations of Nigeria; do not download towns.   
2. Save the downloaded data as points, save it at: E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Nigeria_cities.gpkg 


Requirements: 
1. Return the exact name of the data source as the given names.
2. If a data source

## Pick up the data source handbook

In [13]:
from IPython.display import clear_output

chunks = []

async for chunk in model.astream(source_select_prompt_str):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

# clear_output(wait=True)
clear_output(wait=False)
LLM_reply_str = helper.convert_chunks_to_str(chunks=chunks)

print("Select the data source: \n")
print(LLM_reply_str)

Select the data source: 

{'Explanation': "According to the use requests to download the city locations of Nigeria, I should download data from OpenStreetMap.", "Selected data source": 'OpenStreetMap'}


## Generate the data fetching program

In [14]:
import ast
select_source = ast.literal_eval(LLM_reply_str)

selected_data_source= select_source['Selected data source']
data_source_ID = constants.data_source_dict[selected_data_source]['ID']

print("selected_data_source:", selected_data_source)
print("data_source_ID:", data_source_ID)

handbook_list = constants.handbooks[f"{data_source_ID}"]
handbook_str =  '\n'.join([f"{idx + 1}. {line}" for idx, line in enumerate(handbook_list)])
print()
print(f"Handbook:\n{handbook_str}")

selected_data_source: OpenStreetMap
data_source_ID: OpenStreetMap

Handbook:
1. You may use the Overpass API, which is faster than OSMnx library.
2. If you need to download the administrative boundary of only one place from OpenStreetMap, please use a Python package named 'OSMnx' by this code line: `gdf = ox.geocode_to_gdf(place_name)`. This method is fast and recommended. You do not need to use Overpass and then parse the reply.
3. The overall workflow for fetching data within a place's boundary: 1) get the requested place's osm_id by `osm_id = ox.geocode_to_gdf(place_name)['osm_id'][0]`; 2) form the Overpass query. E.g., `f'relation({osm_id}); map_to_area->.rel; way(area.rel)[railway];'` 3) fetch and save the data.
4. The reason of using `ox.geocode_to_gdf()' to get the osm_id is that this function uses Nominatim API to convert the fuzzy place name into a polygon that most match the given place name, rather than exact matching, which may not return the requested polygon.
5. If you ne

In [15]:
# %load_ext autoreload
# %autoreload 2

download_prompt_str = helper.create_download_prompt(task, selected_data_source, handbook_str)

print(download_prompt_str)

Your role: A professional Python programmer in geographic information science (GIScience). You have worked on GIScience for more than 20 years and know every detail and pitfall when collecting data and coding. You know which websites you can get suitable spatial data and know the methods or tricks to download data, such as OpenStreetMap, Census Bureau, or various APIs. You are also experienced in processing the downloaded data, including saving them in suitable formats, map projections, and creating detailed and useful meta-data. When downloading geo-spatial data, the technical handbook for a particular data source is provided; you can follow it, and write Python code carefully to download the data. 
 
Your mission: download geo-spatial data from the given data source for this task: 1. Download the city locations of Nigeria; do not download towns.   
2. Save the downloaded data as points, save it at: E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Dow

In [16]:
from IPython.display import clear_output

chunks = []

async for chunk in model.astream(download_prompt_str):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

# clear_output(wait=True)
clear_output(wait=False)
LLM_reply_str = helper.convert_chunks_to_str(chunks=chunks)
print(LLM_reply_str)

```python
import geopandas as gpd
import osmnx as ox
import requests
from shapely.geometry import Point

def download_data():
    # Step 1: Get boundary of Nigeria
    place_name = 'Nigeria'
    nigeria_boundary = ox.geocode_to_gdf(place_name)
    
    # Step 2: Get osm_id for the place (Nigeria)
    osm_id = nigeria_boundary['osm_id'].values[0]
    
    # Step 3: Define Overpass API query to get city locations, excluding towns
    overpass_url = "https://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    area({osm_id})->.searchArea;
    (
      node[place=city](area.searchArea);
    );
    out body;
    """
    
    # Step 4: Send request to Overpass API
    response = requests.get(overpass_url, params={'data': overpass_query})
    response.raise_for_status()  # Automatically raises an error for bad status codes
    data = response.json()
    
    # Step 5: Parse the JSON response
    property_list = []
    geometry_list = []
    for element in data['elemen

In [17]:
code = helper.extract_code_from_str(LLM_reply_str, task)
display(Code(code, language='python'))

import geopandas as gpd
import osmnx as ox
import requests
from shapely.geometry import Point

def download_data():
    # Step 1: Get boundary of Nigeria
    place_name = 'Nigeria'
    nigeria_boundary = ox.geocode_to_gdf(place_name)
    
    # Step 2: Get osm_id for the place (Nigeria)
    osm_id = nigeria_boundary['osm_id'].values[0]
    
    # Step 3: Define Overpass API query to get city locations, excluding towns
    overpass_url = "https://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    area({osm_id})->.searchArea;
    (
      node[place=city](area.searchArea);
    );
    out body;
    """
    
    # Step 4: Send request to Overpass API
    response = requests.get(overpass_url, params={'data': overpass_query})
    response.raise_for_status()  # Automatically raises an error for bad status codes
    data = response.json()
    
    # Step 5: Parse the JSON response
    property_list = []
    geometry_list = []
    for element in data['elements']:
    

import geopandas as gpd
import osmnx as ox
import requests
from shapely.geometry import Point

def download_data():
    # Step 1: Get boundary of Nigeria
    place_name = 'Nigeria'
    nigeria_boundary = ox.geocode_to_gdf(place_name)
    
    # Step 2: Get osm_id for the place (Nigeria)
    osm_id = nigeria_boundary['osm_id'].values[0]
    
    # Step 3: Define Overpass API query to get city locations, excluding towns
    overpass_url = "https://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    area({osm_id})->.searchArea;
    (
      node[place=city](area.searchArea);
    );
    out body;
    """
    
    # Step 4: Send request to Overpass API
    response = requests.get(overpass_url, params={'data': overpass_query})
    response.raise_for_status()  # Automatically raises an error for bad status codes
    data = response.json()
    
    # Step 5: Parse the JSON response
    property_list = []
    geometry_list = []
    for element in data['elements']:
        # Create Point geometries from latitude and longitude
        point = Point(element['lon'], element['lat'])
        geometry_list.append(point)
        
        # Extract properties
        properties = {key: element[key] for key in element if key not in {'lat', 'lon'} }
        property_list.append(properties)
    
    # Step 6: Create GeoDataFrame from properties and geometry
    gdf = gpd.GeoDataFrame(property_list, geometry=geometry_list)
    gdf.crs = 'EPSG:4326'
    
    # Step 7: Save to GeoPackage
    output_file = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Nigeria_cities.gpkg"
    gdf.to_file(output_file, layer='nigeria_cities', driver='GPKG')

download_data()

# Execute the generated program

In [18]:
# %autoreload 2
# %load_ext autoreload

code = code.replace('area({osm_id})->.searchArea;', 'relation({osm_id}); map_to_area->.searchArea;')  # GPT-4o never follow the related instruction!
    
code = helper.execute_complete_program(code=code, try_cnt=10, task=task, model_name=model_name, handbook_str=handbook_str)

code = code.replace('area({osm_id})->.searchArea;', 'relation({osm_id}); map_to_area->.searchArea;')  # GPT-4o never follow the related instruction!


display(Code(code, language='python'))




-------------- Running code (trial # 1/10) --------------




--------------- Done ---------------




import geopandas as gpd
import osmnx as ox
import requests
from shapely.geometry import Point

def download_data():
    # Step 1: Get boundary of Nigeria
    place_name = 'Nigeria'
    nigeria_boundary = ox.geocode_to_gdf(place_name)
    
    # Step 2: Get osm_id for the place (Nigeria)
    osm_id = nigeria_boundary['osm_id'].values[0]
    
    # Step 3: Define Overpass API query to get city locations, excluding towns
    overpass_url = "https://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    relation({osm_id}); map_to_area->.searchArea;
    (
      node[place=city](area.searchArea);
    );
    out body;
    """
    
    # Step 4: Send request to Overpass API
    response = requests.get(overpass_url, params={'data': overpass_query})
    response.raise_for_status()  # Automatically raises an error for bad status codes
    data = response.json()
    
    # Step 5: Parse the JSON response
    property_list = []
    geometry_list = []
    for element in data['elements']:
        # Create Point geometries from latitude and longitude
        point = Point(element['lon'], element['lat'])
        geometry_list.append(point)
        
        # Extract properties
        properties = {key: element[key] for key in element if key not in {'lat', 'lon'} }
        property_list.append(properties)
    
    # Step 6: Create GeoDataFrame from properties and geometry
    gdf = gpd.GeoDataFrame(property_list, geometry=geometry_list)
    gdf.crs = 'EPSG:4326'
    
    # Step 7: Save to GeoPackage
    output_file = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Nigeria_cities.gpkg"
    gdf.to_file(output_file, layer='nigeria_cities', driver='GPKG')

download_data()

# Show the downloaded data

In [19]:
ext_name = saved_fname[-4:]
print("Saved file:", saved_fname)
print("ext_name:", ext_name)

Saved file: E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Downloaded_Data\Nigeria_cities.gpkg
ext_name: gpkg


In [21]:
# import leafmap.leafmap as leafmap
import leafmap.foliumap as leafmap

m = leafmap.Map(google_map="HYBRID")

if ext_name == 'gpkg':
    gdf = gpd.read_file(saved_fname)  
    bounding_box = gdf.total_bounds
    # m = leafmap.Map()     
    m.add_gdf(gdf, layer_name="Downloaded_data")
    m.fit_bounds([[bounding_box[1], bounding_box[0]], [bounding_box[3], bounding_box[2]]])

if selected_data_source == 'OpenTopography':
    # m = leafmap.Map()
    m.add_raster(source=saved_fname, layer_name='OpenTopography DEM', colormap='viridis') 

m

In [23]:
if ext_name == '.csv':
    df = pd.read_csv(saved_fname, encoding='ISO-8859-1')
    results = df

if ext_name == 'gpkg':
    gdf = gpd.read_file(saved_fname)
    results = gdf
    # gdf.plot()
    # gdf.explore()

if selected_data_source == 'ESRI_world_imagery':
    results = Image.open(saved_fname)
    
results

,type,id,tags,geometry
0,node,27565036,"{""GNS:dsg_code"": ""PPL"", ""GNS:dsg_string"": ""pop...",POINT (3.93154 7.84105)
1,node,27565054,"{""is_in"": ""Nigeria"", ""is_in:country"": ""Nigeria...",POINT (4.25000 8.13300)
2,node,27565065,"{""admin_level"": ""4"", ""capital"": ""4"", ""is_in:co...",POINT (7.01885 4.76766)
3,node,27565066,"{""admin_level"": ""4"", ""capital"": ""4"", ""name"": ""...",POINT (3.89699 7.37861)
4,node,27565089,"{""admin_level"": ""4"", ""capital"": ""4"", ""is_in:co...",POINT (4.54805 8.49637)
...,...,...,...,...
63,node,5232601618,"{""is_in:country"": ""Nigeria"", ""name"": ""Onitsha""...",POINT (6.80191 6.14616)
64,node,9127704912,"{""name"": ""Ab\u1eb9\u0301\u00f2k\u00fata"", ""nam...",POINT (3.34673 7.15471)
65,node,9643470157,"{""name"": ""Awe"", ""place"": ""city""}",POINT (9.14637 8.11181)
66,node,9702599529,"{""name"": ""Warri"", ""name:en"": ""Warri"", ""name:ru...",POINT (5.74802 5.51862)


In [ ]:
### If you do not like to install Leafmap or have difficulty to install it, 
### just use gdf.explore() to check the downloaded data in an interative way
# gdf.explore()

In [ ]:
STOP